![qiskit_header.png](../../images/qiskit_header.png)

# Quantum Tomography

## Introduction

Quantum tomography is an experimental procedure to reconstruct a description of part of quantum system from the measurement outcomes of a specific set of experiments. In Qiskit we implement the following types of tomography:

1. **Quantum state tomography**: Given a state-preparation circuit that prepares a system in a state, reconstruct a description of the density matrix $\rho$ of the actual state obtained in the system.
2. **Quantum process tomograhpy**: Given a circuit, reconstruct a description of the quantum channel $\mathcal{E}$ that describes the circuit's operator when running on the system.
3. **Quantum gate set tomography**: Performs process tomography on a set of gates in a self-consistant manner, meaning quantum noises on gates used by the tomography process itself is also taken into account.

This notebook gives examples for how to use the ``ignis.verification.tomography`` modules.

In [1]:
# Needed for functions
import numpy as np
import time
from copy import deepcopy

# Import Qiskit classes
import qiskit 
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister, Aer
from qiskit.quantum_info import state_fidelity, process_fidelity
from qiskit.providers.aer import noise
from qiskit.compiler import assemble

# Tomography functions
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.ignis.verification.tomography import process_tomography_circuits, ProcessTomographyFitter
from qiskit.ignis.verification.tomography import gateset_tomography_circuits, GatesetTomographyFitter
import qiskit.ignis.mitigation.measurement as mc

# Auxiliary methods
from qiskit.quantum_info import Choi, Kraus
from qiskit.extensions import HGate, XGate

## Initial examples

### 2-Qubit state tomography Example

In the below example we want to perform state tomography on a 2Q Bell state between qubits 3 and 5. To make the reference circuit we generate the expected statevector using ``statevector_simulator`` between qubits 0 and 1. 

In [2]:
# Create the expected density matrix
q2 = QuantumRegister(2)
bell = QuantumCircuit(q2)
bell.h(q2[0])
bell.cx(q2[0], q2[1])
print(bell)

job = qiskit.execute(bell, Aer.get_backend('statevector_simulator'))
psi_bell = job.result().get_statevector(bell)
print(psi_bell)

      ┌───┐     
q0_0: ┤ H ├──■──
      └───┘┌─┴─┐
q0_1: ─────┤ X ├
           └───┘
[0.70710678+0.j 0.        +0.j 0.        +0.j 0.70710678+0.j]


In [3]:
# Create the actual circuit 
q2 = QuantumRegister(6)
bell = QuantumCircuit(q2)
bell.h(q2[3])
bell.cx(q2[3], q2[5])
print(bell)

                
q3_0: ──────────
                
q3_1: ──────────
                
q3_2: ──────────
      ┌───┐     
q3_3: ┤ H ├──■──
      └───┘  │  
q3_4: ───────┼──
           ┌─┴─┐
q3_5: ─────┤ X ├
           └───┘


Here we are going to generate and run the state tomography circuits. By only passing in the 2 registers we want to measure the state tomography will only run on that reduced $2^2$ Hilbert space. However, if we pass the whole register in the state tomography module will try and fit the full $2^6$ space.

In [4]:
# Generate circuits and run on simulator
t = time.time()
# Generate the state tomography circuits. Only pass in the 
# registers we want to measure (in this case 3 and 5)
qst_bell = state_tomography_circuits(bell, [q2[3],q2[5]])
job = qiskit.execute(qst_bell, Aer.get_backend('qasm_simulator'), shots=5000)
print('Time taken:', time.time() - t)

tomo_bell = StateTomographyFitter(job.result(), qst_bell)

Time taken: 0.028922557830810547


The fitter will output a density matrix ordered according to how we passed in the registers to ``state_tomography_circuits``. 

In [5]:
# Perform the tomography fit
# which outputs a density matrix
rho_bell = tomo_bell.fit()
F_bell = state_fidelity(psi_bell, rho_bell, validate=False)
print('Fit Fidelity =', F_bell)

FATAL: Cannot solve SDPs with > 2x2 matrices without linked blas+lapack libraries
Install blas+lapack and re-compile SCS with blas+lapack library locations
ERROR: init_cone failure
Failure:could not initialize work
Fit Fidelity = 0.9960151819931408


### Repeat the Example with Measurement Noise

In [6]:
#Add measurement noise
noise_model = noise.NoiseModel()
for qi in range(6):
    read_err = noise.errors.readout_error.ReadoutError([[0.75, 0.25],[0.1,0.9]])
    noise_model.add_readout_error(read_err,[qi])
    
#generate the calibration circuits
meas_calibs, state_labels = mc.complete_meas_cal(qubit_list=[3,5])

backend = Aer.get_backend('qasm_simulator')
job_cal = qiskit.execute(meas_calibs, backend=backend, shots=15000, noise_model=noise_model)
job_tomo = qiskit.execute(qst_bell, backend=backend, shots=15000, noise_model=noise_model)

meas_fitter = mc.CompleteMeasFitter(job_cal.result(),state_labels)

tomo_bell = StateTomographyFitter(job_tomo.result(), qst_bell)

#no correction
rho_bell = tomo_bell.fit()
F_bell = state_fidelity(psi_bell, rho_bell, validate=False)
print('Fit Fidelity (no correction) =', F_bell)

#correct data
correct_tomo_results = meas_fitter.filter.apply(job_tomo.result(), method='least_squares')
tomo_bell = StateTomographyFitter(correct_tomo_results, qst_bell)
rho_bell = tomo_bell.fit()
F_bell = state_fidelity(psi_bell, rho_bell, validate=False)
print('Fit Fidelity (w/ correction) =', F_bell)

Fit Fidelity (no correction) = 0.5728099462585374
Fit Fidelity (w/ correction) = 0.9756857354884931


## 1-qubit process tomography example

In [7]:
# Process tomography of a Hadamard gate
q = QuantumRegister(1)
circ = QuantumCircuit(q)
circ.h(q[0])

# Run circuit on unitary simulator to find ideal unitary
job = qiskit.execute(circ, Aer.get_backend('unitary_simulator'))
ideal_unitary = job.result().get_unitary(circ)

# convert to Choi-matrix in column-major convention
vector_ideal = ideal_unitary.ravel(order='F')
choi_ideal = np.outer(vector_ideal, vector_ideal.conj())

# Generate process tomography circuits and run on qasm simulator
qpt_circs = process_tomography_circuits(circ, q)
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=4000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_tomo = ProcessTomographyFitter(job.result(), qpt_circs)
qpt_tomo.data

{(('Zp',), ('X',)): {'0': 4000},
 (('Zp',), ('Y',)): {'0': 1998, '1': 2002},
 (('Zp',), ('Z',)): {'0': 2047, '1': 1953},
 (('Zm',), ('X',)): {'1': 4000},
 (('Zm',), ('Y',)): {'0': 1996, '1': 2004},
 (('Zm',), ('Z',)): {'0': 1990, '1': 2010},
 (('Xp',), ('X',)): {'0': 1978, '1': 2022},
 (('Xp',), ('Y',)): {'0': 1992, '1': 2008},
 (('Xp',), ('Z',)): {'0': 4000},
 (('Yp',), ('X',)): {'0': 1996, '1': 2004},
 (('Yp',), ('Y',)): {'1': 4000},
 (('Yp',), ('Z',)): {'0': 1991, '1': 2009}}

In [8]:
# Tomographic reconstruction
t = time.time()
choi_lstsq = qpt_tomo.fit(method='auto')
print('fit time:', time.time() - t)
print('fit fidelity (state):', state_fidelity(choi_ideal / 2, choi_lstsq.data / 2, validate=False))
print('fit fidelity (process):', np.real(process_fidelity(choi_ideal, choi_lstsq.data, require_cp=False)))

FATAL: Cannot solve SDPs with > 2x2 matrices without linked blas+lapack libraries
Install blas+lapack and re-compile SCS with blas+lapack library locations
ERROR: init_cone failure
Failure:could not initialize work
fit time: 0.07278251647949219
fit fidelity (state): 0.9958746339699709
fit fidelity (process): 0.9917662844081608


## 1-qubit process tomography of two-qubit swap gate

We will prepare qubit-0 and measure qubit-1 so the reconstructed channel should be an identity.

In [9]:
# Process tomography of a Hadamard gate
q = QuantumRegister(2)
circ = QuantumCircuit(q)
circ.swap(q[0], q[1])

# Ideal channel is a unitary
ideal_unitary = np.eye(2)
vector_ideal = ideal_unitary.ravel(order='F')
choi_ideal = np.outer(vector_ideal, vector_ideal.conj())

# Generate process tomography circuits and run on qasm simulator
# We use the optional prepared_qubits kwarg to specify that the prepared qubit was different to measured qubit
qpt_circs = process_tomography_circuits(circ, q[1], prepared_qubits=q[0])
job = qiskit.execute(qpt_circs, Aer.get_backend('qasm_simulator'), shots=2000)

# Extract tomography data so that counts are indexed by measurement configuration
qpt_tomo = ProcessTomographyFitter(job.result(), qpt_circs)
qpt_tomo.data

{(('Zp',), ('X',)): {'0': 966, '1': 1034},
 (('Zp',), ('Y',)): {'0': 1023, '1': 977},
 (('Zp',), ('Z',)): {'0': 2000},
 (('Zm',), ('X',)): {'0': 993, '1': 1007},
 (('Zm',), ('Y',)): {'0': 1009, '1': 991},
 (('Zm',), ('Z',)): {'1': 2000},
 (('Xp',), ('X',)): {'0': 2000},
 (('Xp',), ('Y',)): {'0': 994, '1': 1006},
 (('Xp',), ('Z',)): {'0': 981, '1': 1019},
 (('Yp',), ('X',)): {'0': 997, '1': 1003},
 (('Yp',), ('Y',)): {'0': 2000},
 (('Yp',), ('Z',)): {'0': 1004, '1': 996}}

In [10]:
# Tomographic reconstruction

t = time.time()
choi = qpt_tomo.fit(method='auto')
print('fit time:', time.time() - t)
print('fit fidelity (state):', state_fidelity(choi_ideal / 2, choi.data / 2, validate=False))
print('fit fidelity (process):', np.real(process_fidelity(choi_ideal, choi.data, require_cp=False)))

fit time: 0.003989458084106445
fit fidelity (state): 0.9931039890728947
fit fidelity (process): 0.9862555324154729


## Advances examples

## Generating and fitting random states

We now test the functions on the state generated by a circuit consisting of a layer of random single qubit unitaries u3.

In [11]:
def random_u3_tomo(nq, shots):
    
    def rand_angles():
        return tuple(2 * np.pi * np.random.random(3) - np.pi)
    q = QuantumRegister(nq)
    circ = QuantumCircuit(q)
    for j in range(nq):
        circ.u3(*rand_angles(), q[j])
    job = qiskit.execute(circ, Aer.get_backend('statevector_simulator'))
    psi_rand = job.result().get_statevector(circ)
    
    qst_circs = state_tomography_circuits(circ, q)
    job = qiskit.execute(qst_circs, Aer.get_backend('qasm_simulator'),
                         shots=shots)
    tomo_data = StateTomographyFitter(job.result(), qst_circs)
    rho = tomo_data.fit(method='auto')
    
    print('F fit =', state_fidelity(psi_rand, rho, validate=False))

In [12]:
for j in range(5):
    print('Random single-qubit unitaries: set {}'.format(j))
    random_u3_tomo(3, 5000)

Random single-qubit unitaries: set 0
F fit = 0.9912624488590052
Random single-qubit unitaries: set 1
F fit = 0.9955321652578383
Random single-qubit unitaries: set 2
F fit = 0.9955341807191802
Random single-qubit unitaries: set 3
F fit = 0.9961116923414528
Random single-qubit unitaries: set 4
F fit = 0.9915573599156398


## 5-Qubit Bell State

In [13]:
# Create a state preparation circuit
q5 = QuantumRegister(5)
bell5 = QuantumCircuit(q5)
bell5.h(q5[0])
for j in range(4):
    bell5.cx(q5[j], q5[j + 1])

# Get ideal output state
job = qiskit.execute(bell5, Aer.get_backend('statevector_simulator'))
psi_bell5 = job.result().get_statevector(bell5)

# Generate circuits and run on simulator
t = time.time()
qst_bell5 = state_tomography_circuits(bell5, q5)
job = qiskit.execute(qst_bell5, Aer.get_backend('qasm_simulator'), shots=5000)

# Extract tomography data so that counts are indexed by measurement configuration
tomo_bell5 = StateTomographyFitter(job.result(), qst_bell5)
print('Time taken:', time.time() - t)

Time taken: 6.197464942932129


In [14]:
t = time.time()
rho_bell5 = tomo_bell5.fit(method='auto')
print('Time taken:', time.time() - t)
print('Fit Fidelity:', state_fidelity(psi_bell5, rho_bell5, validate=False))

Time taken: 3.1012156009674072
Fit Fidelity: 0.9936014720187993


## 2-Qubit Conditional State Tomography 

In this example, we have a three-qubit system where one of the qubits will be an ancilla for performing state tomography, i.e. only perform tomography when the third qubit is in the state "1". The circuit is setup in such a way that after conditional tomography we will get a Bell state on the first two qubits.

First make a 3Q GHZ state with no classical measurements.

In [15]:
# Create the actual circuit 
q2 = QuantumRegister(3)
ghz = QuantumCircuit(q2)
ghz.h(q2[0])
ghz.cx(q2[0], q2[1])
ghz.cx(q2[1], q2[2])
ghz.h(q2[2])
print(ghz)

         ┌───┐               
q2029_0: ┤ H ├──■────────────
         └───┘┌─┴─┐          
q2029_1: ─────┤ X ├──■───────
              └───┘┌─┴─┐┌───┐
q2029_2: ──────────┤ X ├┤ H ├
                   └───┘└───┘


Here we are going to generate and run the state tomography circuits. Only pass the registers we want to perform state tomography on. The code will generate a new classical register for only those measurements.

In [16]:
qst_ghz = state_tomography_circuits(ghz, [q2[0],q2[1]])
print(qst_ghz[0])

         ┌───┐                ░ ┌───┐┌─┐   
q2029_0: ┤ H ├──■─────────────░─┤ H ├┤M├───
         └───┘┌─┴─┐           ░ ├───┤└╥┘┌─┐
q2029_1: ─────┤ X ├──■────────░─┤ H ├─╫─┤M├
              └───┘┌─┴─┐┌───┐ ░ └───┘ ║ └╥┘
q2029_2: ──────────┤ X ├┤ H ├─░───────╫──╫─
                   └───┘└───┘ ░       ║  ║ 
  c10: 2/═════════════════════════════╩══╩═
                                      0  1 


Now make a copy of this circuit (we will need it for the fitter) and make a new circuit with an ancilla measurement attached (this is what will be run):

In [17]:
#Make a copy without the ancilla register
qst_ghz_no_anc = deepcopy(qst_ghz)
ca = ClassicalRegister(1)
for qst_ghz_circ in qst_ghz:
    qst_ghz_circ.add_register(ca)
    qst_ghz_circ.measure(q2[2],ca[0])

In [18]:
#Run in Aer
job = qiskit.execute(qst_ghz, Aer.get_backend('qasm_simulator'), shots=10000)
raw_results = job.result()

Before sending the results to the state tomography fitter we must strip the register for the Q2 measurement and only keep the results when that register is 1.

In [19]:

new_result = deepcopy(raw_results)

for resultidx, _ in enumerate(raw_results.results):
    old_counts = raw_results.get_counts(resultidx)
    new_counts = {}
    
    #change the size of the classical register
    new_result.results[resultidx].header.creg_sizes = [new_result.results[resultidx].header.creg_sizes[0]]
    new_result.results[resultidx].header.clbit_labels = new_result.results[resultidx].header.clbit_labels[0:-1]
    new_result.results[resultidx].header.memory_slots = 2
    
    for reg_key in old_counts:
        reg_bits = reg_key.split(' ')
        if reg_bits[0]=='1':
            new_counts[reg_bits[1]]=old_counts[reg_key]

        new_result.results[resultidx].data.counts = new_counts 

In [20]:
tomo_bell = StateTomographyFitter(new_result, qst_ghz_no_anc)
# Perform the tomography fit
# which outputs a density matrix
rho_bell = tomo_bell.fit()

In [21]:
np.around(rho_bell,3)

array([[ 0.5  +0.j   , -0.001-0.005j, -0.005+0.005j, -0.499-0.005j],
       [-0.001+0.005j,  0.   +0.j   , -0.   -0.j   ,  0.002-0.005j],
       [-0.005-0.005j, -0.   +0.j   ,  0.   +0.j   ,  0.004+0.005j],
       [-0.499+0.005j,  0.002+0.005j,  0.004-0.005j,  0.499+0.j   ]])

# Gate set tomography

## 1-Qubit gate set tomography Examples

The main difference between gate set tomography and process tomography is that in gate set tomography, the input consists of a gate set basis: A set of gates that are both used in the initialization/measurement phase of the tomography, and are being reconstructed.

Qiskit supplies a default gateset basis; in order to use this gateset basis in order to reconstruct another gate, this gate should be added to the basis. We use the following method to simplify the process:

In [22]:
from qiskit.ignis.verification.tomography.basis import default_gateset_basis

def collect_tomography_data(shots=10000,
                            noise_model=None,
                            gateset_basis='Standard GST'):
    backend_qasm = Aer.get_backend('qasm_simulator')
    circuits = gateset_tomography_circuits(gateset_basis=gateset_basis)
    qobj = assemble(circuits, shots=shots)
    result = backend_qasm.run(qobj, noise_model=noise_model).result()
    fitter = GatesetTomographyFitter(result, circuits, gateset_basis)
    return fitter

def gate_set_tomography(gate, noise_model=None):
    basis = default_gateset_basis()
    basis.add_gate(gate)
    fitter = collect_tomography_data(shots=10000, noise_model=noise_model, gateset_basis=basis)
    result_gates = fitter.fit()
    result_gate = result_gates[gate.name]
    return Choi(result_gate)

### Noiseless 1-qubit gate set tomography

In [23]:
choi_ideal = Choi(HGate()).data
t = time.time()
choi = gate_set_tomography(HGate())
print('fit time:', time.time() - t)
print('fit fidelity (state):', state_fidelity(choi_ideal / 2, choi.data / 2, validate=False))
print('fit fidelity (process):', np.real(process_fidelity(choi_ideal, choi.data, require_cp=False)))

fit time: 2.4005630016326904
fit fidelity (state): 1.0040070587253802
fit fidelity (process): 1.0080301736826522


### 1-qubit gate set tomography with amplitude damping noise

Note that we add the noise to all gates (meaning also to the gates performing the tomography)

In [24]:
gamma = 0.1
A0 = [[1, 0], [0, np.sqrt(1 - gamma)]]
A1 = [[0, np.sqrt(gamma)], [0, 0]]
noise_choi = Choi(Kraus([A0, A1])).data
error = noise.errors.QuantumError([([{'name': 'kraus',
                         'qubits': [0],
                         'params': [A0, A1]}], 1)])
noise_model = noise.NoiseModel()
noise_model.add_all_qubit_quantum_error(error, ['u1', 'u2', 'u3', 'x'])

In [25]:
choi_ideal = (Choi(XGate()).compose(noise_choi)).data
t = time.time()
choi = gate_set_tomography(XGate(), noise_model)
print('fit time:', time.time() - t)
print('fit fidelity (state):', state_fidelity(choi_ideal / 2, choi.data / 2, validate=False))
print('fit fidelity (process):', np.real(process_fidelity(choi_ideal, choi.data, require_cp=False)))

fit time: 1.9352388381958008
fit fidelity (state): 0.9947735751075302
fit fidelity (process): 0.8362664609647777


In [26]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright